In [2]:
import load_files as lf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
file_path = "/Users/blazejmanczak/Desktop/Q1/DataChallenge3/"
flow_data_DRU, level_data_DRU = lf.get_measurements(file_path +"/waterschap-aa-en-maas_sewage_2019/sewer_data/data_pump/RG8150")
flow_data_HAA, level_data_HAA = lf.get_measurements(file_path +"/waterschap-aa-en-maas_sewage_2019/sewer_data/data_pump/RG8170")
rain_data = lf.get_rain(file_path +"/waterschap-aa-en-maas_sewage_2019/sewer_data/rain_timeseries")
data = lf.sdf(file_path +"/waterschap-aa-en-maas_sewage_2019/sewer_model/aa-en-maas_sewer_shp")

AttributeError: 'NoneType' object has no attribute 'is_empty'

In [38]:
data.area_data["village_ID"] = data.area_data["sewer_system"].str.slice(4,7)
#area_names = rain_data.columns[3:]

area_data_DRU = data.area_data.loc[data.area_data["village_ID"] == "DRU"]
area_data_HAA = data.area_data.loc[data.area_data["village_ID"] == "HAA"]

In [55]:
# Areas belonging to specific pumps that are in the rain dataset
DRU_areas = area_data_DRU["area_name"][area_data_DRU["area_name"].apply(lambda i: i in rain_data.columns)].to_list()
HAA_areas = area_data_HAA["area_name"][area_data_HAA["area_name"].apply(lambda i: i in rain_data.columns)].to_list()

In [85]:
DRU_rain = rain_data.loc[:, ["Start", "End"] + DRU_areas]
HAA_rain = rain_data.loc[:, ["Start", "End"] + HAA_areas]

DRU_rain["Start"] = pd.to_datetime(DRU_rain["Start"])
DRU_rain["Date"] = DRU_rain["Start"].apply(lambda i: i.date())
DRU_rain["Total"] = DRU_rain.iloc[:, 2:].sum(axis=1)
DRU_rain = DRU_rain.groupby("Date")["Total"].sum().reset_index(drop=False)

In [ ]:
level_data_DRU["TimeStamp"] = pd.to_datetime(level_data_DRU["TimeStamp"])
flow_data_DRU["TimeStamp"] = pd.to_datetime(flow_data_DRU["TimeStamp"])

In [95]:
level_data_DRU["Date"] = level_data_DRU["TimeStamp"].apply(lambda i: i.date())
level_data_DRU = level_data_DRU.groupby("Date")["Value"].mean().reset_index(drop=False)

In [112]:
flow_data_DRU["Date"] = flow_data_DRU["TimeStamp"].apply(lambda i: i.date())
flow_data_DRU = flow_data_DRU.groupby("Date")["Value"].mean().reset_index(drop=False)

In [113]:
DRU_rain_level = pd.merge(level_data_DRU, DRU_rain, on="Date")
DRU_rain_flow = pd.merge(flow_data_DRU, DRU_rain, on="Date")

In [114]:
DRU_rain_level.to_csv("heya.csv")
DRU_rain_flow.to_csv("heya_flow.csv")